In [12]:
"""This simple notebook demonstrates the workflow of using the TensorFlow converter.
"""
import sys, os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data # Import MINST data
freeze_tool_path = tf.__path__[0]+'/python/tools/'
sys.path.append(freeze_tool_path)
from freeze_graph import freeze_graph

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
image_input:  Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
pred output: Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Epoch: 0001 cost= 1.183640801
Epoch: 0002 cost= 0.665039274
Epoch: 0003 cost= 0.552708297
Epoch: 0004 cost= 0.498620908
Epoch: 0005 cost= 0.465537337
Epoch: 0006 cost= 0.442588716
Epoch: 0007 cost= 0.425542378
Epoch: 0008 cost= 0.412158466
Epoch: 0009 cost= 0.401340964
Epoch: 0010 cost= 0.392393883
Epoch: 0011 cost= 0.384757366
Epoch: 0012 cost= 0.378174489
Epoch: 0013 cost= 0.372428433
Epoch: 0014 cost= 0.367292162
Epoch: 0015 cost= 0.362664467
Epoch: 0016 cost= 0.358602979
Epoch: 0017 cost= 0.354869082
Epoch: 0018 cost= 0.351424914
Epoch: 0019 cost= 0.348304900
Epoch: 0020 cost= 0.345419539
Epoch: 0021 cost= 0.342728971
Epoch: 0022 cost= 0.340255892
Epoch: 0023 cost= 

In [ ]:
"""
Step 0: Before you run this notebook, view run the example script linear_mnist_train.py
to get a trained TensorFlow network.
This may take a few minutes.
"""
import linear_mnist_train
linear_mnist_train.train()

In [13]:
"""
Step 1: "Freeze" your tensorflow model - convert your TF model into a stand-alone graph definition file
Inputs: 
(1) TensorFlow code
(2) trained weights in a checkpoint file
(3) Input tensors' name to TF model, and
(4) The output tensors' name you want to use in inference
Outputs: 
(1) A frozen TensorFlow GraphDef, with trained weights frozen into it
"""

# Provide these to run freeze_graph:
# Graph definition file, stored as protobuf TEXT
graph_def_file = './model.pbtxt'
# Trained model's checkpoint name
checkpoint_file = './checkpoints/model.ckpt'
# Frozen model's output name
frozen_model_file = './frozen_model.pbtxt'
# Output nodes. If there're multiple output ops, use comma separated string, e.g. "out1,out2".
output_node_names = 'Softmax' 


# Call freeze graph
freeze_graph(input_graph=graph_def_file,
             input_saver="",
             input_binary=False,
             input_checkpoint=checkpoint_file,
             output_node_names=output_node_names,
             restore_op_name="save/restore_all",
             filename_tensor_name="save/Const:0",
             output_graph=frozen_model_file,
             clear_devices=True,
             initializer_nodes="")



INFO:tensorflow:Restoring parameters from ./checkpoints/model.ckpt


INFO:tensorflow:Restoring parameters from ./checkpoints/model.ckpt


INFO:tensorflow:Froze 2 variables.


INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.
8 ops in the final graph.


In [14]:
import tfcoreml

In [15]:
"""
Step 2: Call converter
"""

# Provide these inputs in addition to inputs in Step 1
# A dictionary of input tensors' name and shape (with batch)
input_tensor_shapes = {"Placeholder:0":[1,784]} # batch size is 1
# Output CoreML model path
coreml_model_file = './model.mlmodel'
output_tensor_names = ['Softmax:0']


# Call the converter
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file, 
        mlmodel_path=coreml_model_file, 
        input_name_shape_dict=input_tensor_shapes,
        output_feature_names=output_tensor_names)



Shapes not found for 4 tensors. Executing graph to determine shapes. 
2/8: Converting op name: Variable_1/read ( type:  Identity )
4/8: Converting op name: Variable/read ( type:  Identity )
5/8: Converting op name: Placeholder ( type:  Placeholder )
Skipping name of placeholder
6/8: Converting op name: MatMul ( type:  MatMul )
8/8: Converting op name: Softmax ( type:  Softmax )

 Core ML model generated. Saved at location: ./model.mlmodel 

Core ML input(s):  [('Placeholder:0', Array({784}))]
Core ML output(s):  [('Softmax:0', Array({10}))]


In [16]:
"""
Step 3: Run the converted model
"""

# Provide CoreML model with a dictionary as input. Change ':0' to '__0'
# as Swift / Objective-C code generation do not allow colons in variable names
coreml_inputs = {'Placeholder__0': np.random.rand(1,1,784)} # (sequence_length=1,batch=1,channels=784)
coreml_output = coreml_model.predict(coreml_inputs, useCPUOnly=False)


In [17]:
print coreml_output

{u'Softmax__0': array([  1.87052283e-02,   2.34643790e-06,   1.04608245e-01,
         1.60167947e-01,   1.38722942e-04,   6.70092702e-01,
         3.96387605e-03,   1.85578875e-03,   3.95736843e-02,
         8.91480537e-04])}
